In [131]:
#libraries needed
import pandas as pd


In [132]:
#file locations

crsp_names = 'D:\OneDrive - Northwestern University\My files\data_feeds\CRSP\data\crsp_names.parquet' #crsp names file ticker format root+suffix has cusip8
comp_names = 'D:\OneDrive - Northwestern University\My files\data_feeds\CompSnap\Data\comp_names.parquet'  # compustat names file: ticker format root+.+suffix has cusip8
ibes_names = 'D:\OneDrive - Northwestern University\My files\data_feeds\IBES\Data\ibes_names.parquet' # IBES names file: ticker format root+.+suffix has cusip8
om_names = 'D:\OneDrive - Northwestern University\My files\data_feeds\OptionMetrics\Data\om_names.parquet' # OM names file: ticker format root+.+suffix has cusip8
# rp names file: ticker format root+.+suffix has cusip8
rp_names = r'D:\OneDrive - Northwestern University\My files\data_feeds\RavenPack\data\rp_names.parquet'
# TrueBeats names file: ticker format root+.+suffix has cusip8 and cusip9
truebeats_names = r'D:\OneDrive - Northwestern University\My files\data_feeds\TrueBeats\Data\tb_names.parquet'
# Zacks names file: ticker format root+.+suffix has cusip8 and hcusip
zacks_names = 'D:\OneDrive - Northwestern University\My files\data_feeds\zacks\data\zacks_names.parquet'
# wsh names file: ticker format root+.+suffix has cusip8 (but only later in the period)
wsh_names = 'D:\OneDrive - Northwestern University\My files\data_feeds\WSH\Data\wsh_names.parquet'



In [155]:
# write the function to parse crsp style tickers

def crsp_style(df,symbol):
    y=df
    y[['cusip8']] = y[['cusip8']].astype('string')
    #y[[symbol,'cusip8']] = y[[symbol,'cusip8']].str.upper()
    y[[symbol, 'suffix']] = y[[symbol, 'suffix']].astype('string')
    
    y[[symbol, 'suffix']] = y[[symbol, 'suffix']].fillna('')
    y['root'] = y[symbol]
    
    idx2=y.loc[:,'suffix']==y.loc[:,'root'].str.strip().str[-1]
    y.loc[idx2,'root']=y.loc[idx2,'root'].str[:-1]
    y['crsp_tsym'] = y[symbol]
    y['tsym']=y.root
    y['bbg_tsym']=y.root
    y.loc[idx2,'tsym']=y.loc[idx2,'root']+'.'+y.loc[idx2,'suffix']
    y.loc[idx2,'bbg_tsym']=y.loc[idx2,'root']+'/'+y.loc[idx2,'suffix']
    y['start_date'] = pd.to_datetime(
        y['start_date'], format='%Y%m%d').dt.date
    today = pd.to_datetime('today')
    idx = y['end_date'] >= today
    y.loc[idx,'end_date'] = today
    y['end_date'] = pd.to_datetime(
        y['end_date'], format='%Y%m%d').dt.date
    
    return y

# write the function to parse ibes/'dot' style tickers

def dot_style(df, symbol):
    y = df
    y[[symbol]] = y[[symbol]].astype('string')
    y[['cusip8']] = y[['cusip8']].astype('string')
    #y[[symbol,'cusip8']] = y[[symbol,'cusip8']].str.upper()
    y[['root', 'suffix']] = y[symbol].str.split(
        '[ /.]', n=1, expand=True)
    y[['root', 'suffix']] = y[['root', 'suffix']].fillna('')
    y['tsym'] = y.root+'.'+y.suffix
    y['crsp_tsym'] = y.root+y.suffix
    y['bbg_tsym'] = y.root+'/'+y.suffix
    idx = y.suffix == ''
    y.loc[idx, 'tsym'] = y.loc[idx, 'root']
    y.loc[idx, 'bbg_tsym'] = y.loc[idx, 'root']
    y['start_date'] = pd.to_datetime(
        y['start_date'], format='%Y%m%d').dt.date
    today = pd.to_datetime('today')
    idx = y['end_date'] >= today
    y.loc[idx, 'end_date'] = today
    y['end_date'] = pd.to_datetime(
        y['end_date'], format='%Y%m%d').dt.date


    return y


In [156]:

crsp = crsp_style(pd.read_parquet(crsp_names), 'ticker')
crsp.head()


,permno,permco,ticker,cusip8,suffix,start_date,end_date,root,crsp_tsym,tsym,bbg_tsym
13967,12160.0,3.0,CBNF,14184710,,1977-01-27,1981-04-26,CBNF,CBNF,CBNF,CBNF
26443,12160.0,3.0,AAIR,02136810,,1981-04-27,1982-10-14,AAIR,AAIR,AAIR,AAIR
37039,63845.0,4.0,AALX,01447310,,1972-12-14,1973-05-28,AALX,AALX,AALX,AALX
8822,63845.0,4.0,AALS,01447610,,1973-05-29,1982-01-24,AALS,AALS,AALS,AALS
32154,63845.0,4.0,AAL,01447610,,1982-01-25,1997-02-21,AAL,AAL,AAL,AAL


In [135]:
ibes = dot_style(pd.read_parquet(ibes_names), 'ticker')
zacks = dot_style(pd.read_parquet(zacks_names), 'hticker')
comp = dot_style(pd.read_parquet(comp_names), 'ticker')
om = dot_style(pd.read_parquet(om_names), 'ticker')
tb = dot_style(pd.read_parquet(truebeats_names), 'hticker')
rp = dot_style(pd.read_parquet(rp_names), 'tsym')
wsh=dot_style(pd.read_parquet(wsh_names), 'hticker')


In [136]:
rp




,tsym,cusip8,crsp_tsym,bbg_tsym,rp_id,start_date,end_date,root,suffix
0,000001,Y6896T10,000001,000001,EB6C9A,2000-01-01,2022-03-23,000001,
1,000027,Y7740R10,000027,000027,DD0AA3,2000-01-01,2022-03-23,000027,
2,000030,Y9695N13,000030,000030,33D6AE,2014-11-19,2022-03-23,000030,
3,000031,Y7740B10,000031,000031,86BB2F,2000-01-01,2022-03-23,000031,
4,000039,Y1457J11,000039,000039,3CA333,2000-01-01,2022-03-23,000039,
...,...,...,...,...,...,...,...,...,...
23173,ZYTLF,55941020,ZYTLF,ZYTLF,9D407F,2000-01-01,2022-03-23,ZYTLF,
23174,ZYXI,98986M10,ZYXI,ZYXI,A79D88,2008-07-09,2022-03-23,ZYXI,
23175,ZYY,B0449N10,ZYY,ZYY,19F73C,2007-02-12,2022-03-23,ZYY,
23176,ZZ,81213930,ZZ,ZZ,B77642,2006-06-02,2013-03-18,ZZ,


In [137]:
cols = ['crsp_tsym', 'cusip8', 'start_date', 'end_date']
r1 = pd.concat([crsp[cols], ibes[cols], zacks[cols],
               comp[cols], om[cols], tb[cols], rp[cols], wsh[cols]])
r1 = r1.drop_duplicates().reset_index(drop=True)
r1


,index,crsp_tsym,cusip8,start_date,end_date
0,13967,CBNF,14184710,1977-01-27,1981-04-26
1,26443,AAIR,02136810,1981-04-27,1982-10-14
2,37039,AALX,01447310,1972-12-14,1973-05-28
3,8822,AALS,01447610,1973-05-29,1982-01-24
4,32154,AAL,01447610,1982-01-25,1997-02-21
...,...,...,...,...,...
444238,23770,543396IN,E982J010,2022-03-14,2022-03-14
444239,23771,LFLY,52178J10,2022-03-22,2022-03-22
444240,23772,2351TW,00023510,2022-03-22,2022-03-22
444241,23773,GLS,36850R20,2022-03-22,2022-03-22


In [138]:
min = r1.groupby(by=['crsp_tsym', 'cusip8'])[
    'start_date'].min().reset_index()

max= r1.groupby(by=['crsp_tsym', 'cusip8'])[
    'end_date'].max().reset_index()
max


,crsp_tsym,cusip8,end_date
0,,00035210,1973-06-05
1,,00077410,1975-01-23
2,,00087210,1992-12-14
3,,00087B10,2006-03-19
4,,00088410,1973-03-21
...,...,...,...
235709,ZZZZE,98156G10,2015-12-10
235710,ZZZZN,53399W26,2008-03-12
235711,ZZZZN,81299X23,2014-03-27
235712,ZZZZQ,70355520,2000-10-17


In [139]:
range=pd.merge(min,max,on=['crsp_tsym', 'cusip8'])
idx = (range.crsp_tsym != '') & (range.cusip8 != '')
range1=range.loc[idx,:]

range1 = range1.drop_duplicates()


,crsp_tsym,cusip8,start_date,end_date
2458,000001,Y6896T10,2000-01-01,2022-03-23
2459,000027,Y7740R10,2000-01-01,2022-03-23
2460,000030,Y9695N13,2014-11-19,2022-03-23
2462,000030KR,70000300,2018-05-19,2020-03-18
2463,000031,Y7740B10,2000-01-01,2022-03-23
...,...,...,...,...
235709,ZZZZE,98156G10,2001-10-03,2015-12-10
235710,ZZZZN,53399W26,2006-10-20,2008-03-12
235711,ZZZZN,81299X23,2014-03-27,2014-03-27
235712,ZZZZQ,70355520,1996-01-02,2000-10-17


In [140]:
#now see if there is agreement in the files 

import sqlite3
con = sqlite3.connect("D:\OneDrive - Northwestern University\My files\data_feeds\linking_databases\data\sec_names.db")
range1.to_sql("crsp_based_names", con, index=False, if_exists='replace')

ibes.to_sql("ibes_names", con, index=False, if_exists='replace')
crsp.to_sql("crsp_names", con, index=False, if_exists='replace')
zacks.to_sql("zacks_names", con, index=False, if_exists='replace')
comp.to_sql("comp_names", con, index=False, if_exists='replace')
om.to_sql("om_names", con, index=False, if_exists='replace')
tb.to_sql("tb_names", con, index=False, if_exists='replace')
rp.to_sql("rp_names", con, index=False, if_exists='replace')
wsh.to_sql("wsh_names", con, index=False, if_exists='replace')


In [144]:
#create a local db for queries
sql = """SELECT a.*,(b.permno IS NOT NULL) as crsp,(c.gvkey IS NOT NULL) as comp,
       (d.ibes_id IS NOT NULL) as ibes,(e.secid IS NOT NULL) as om,(f.rp_id IS NOT NULL) as rp,
       (g.hcusip IS NOT NULL) as tb,(h.zid IS NOT NULL) as zacks,(a1.wsh_id IS NOT NULL) as wsh
FROM crsp_based_names  a left join crsp_names as b
on (a.crsp_tsym=b.crsp_tsym and a.cusip8=b.cusip8)
AND ( a.start_date<= b.start_date and  a.end_date>= b.end_date)
left join comp_names as c
on (a.crsp_tsym = c.crsp_tsym and a.cusip8=c.cusip8
    and a.start_date<= c.start_date and  a.end_date>= c.end_date)
left join ibes_names as d
on (a.crsp_tsym = d.crsp_tsym and a.cusip8=d.cusip8
    and a.start_date<= d.start_date and  a.end_date>= d.end_date)
left join om_names as e
on (a.crsp_tsym = e.crsp_tsym and a.cusip8=e.cusip8
    and a.start_date<= e.start_date and  a.end_date>= e.end_date)
left join rp_names as f
on (a.crsp_tsym = f.crsp_tsym and a.cusip8=f.cusip8
    and a.start_date<= f.start_date and  a.end_date>= f.end_date)
left join tb_names as g
on (a.crsp_tsym = g.crsp_tsym and a.cusip8=g.cusip8
    and a.start_date<= g.start_date and  a.end_date>= g.end_date)
left join zacks_names as h
on (a.crsp_tsym = h.crsp_tsym and a.cusip8=h.cusip8
    and a.start_date<= h.start_date and  a.end_date>= h.end_date)
left join wsh_names as a1
on (a.crsp_tsym = a1.crsp_tsym and a.cusip8=a1.cusip8
    and a.start_date<= a1.start_date and  a.end_date>= a1.end_date)
order  by a.crsp_tsym,a.cusip8,a.start_date,a.end_date;"""
matches = pd.read_sql_query(sql, con)


In [145]:
matches
cols = [ 'crsp', 'comp', 'ibes',        'om', 'rp', 'tb', 'zacks','wsh']
matches['n_matched'] = matches[cols].sum(axis=1)



In [146]:


cmatched = matches[matches.n_matched>=2]
cmatched = cmatched.drop_duplicates()
cmatched = cmatched.sort_values(
    by='n_matched', ascending=False, ignore_index=True)
cmatched.head(200)


,crsp_tsym,cusip8,start_date,end_date,crsp,comp,ibes,om,rp,tb,zacks,wsh,n_matched
0,PG,74271810,1968-01-02,2030-12-31,1,1,1,1,1,1,1,1,8
1,MSON,60487110,1994-01-18,2030-12-31,1,1,1,1,1,1,1,1,8
2,OMIC,82933R10,2021-05-25,2030-12-31,1,1,1,1,1,1,1,1,8
3,USAP,91383710,1994-12-14,2030-12-31,1,1,1,1,1,1,1,1,8
4,DUOL,26603R10,2021-07-22,2030-12-31,1,1,1,1,1,1,1,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,MSGM,62011B10,2021-01-07,2030-12-31,1,1,1,1,1,1,1,1,8
196,OMC,68191910,1990-06-27,2030-12-31,1,1,1,1,1,1,1,1,8
197,OMCL,68213N10,2000-01-01,2030-12-31,1,1,1,1,1,1,1,1,8
198,PLMR,69753M10,2019-04-10,2030-12-31,1,1,1,1,1,1,1,1,8
